In [173]:
from pyTigerGraph import TigerGraphConnection
import pyTigerGraph as tg
import json
import torch
import numpy as np
import pandas as pd
import warnings
import torch
import torch.nn.functional as F
from pyTigerGraph.gds.metrics import Accumulator, Accuracy
import matplotlib.pyplot as plt
import random

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
def connection():
    with open("../../configs/tigergraph_config_1.json", "r") as config:
        args = json.load(config)
    
    conn = tg.TigerGraphConnection(
        host=args["host"],
        graphname=args["graphname"],
        username=args["username"],
        password=args["password"],
        gsqlSecret=args["gsqlSecret"],
        certPath=args["certPath"]
    )

    if not connected(conn):
        raise ConnectionError("Failed to connect to GSQL")
    return conn

def connected(conn):
    if not conn.echo() == "Hello GSQL":
        return False
    return True

In [53]:
conn = connection()

In [25]:
# https://dev.tigergraph.com/forum/t/tigergraph-python-connection-issue/2776
# echo | openssl s_client  -connect 140ae89e8e714a18a7a51df2beca1811.i.tgcloud.io:443 |  sed -ne '/-BEGIN CERTIFICATE-/,/-END CERTIFICATE-/p' > 'C:\Users\ryand\DSC180B\dsc180b-project\my-cert.txt'
# have to put my-cert.txt in C:\User\ryand\.gsql\my-cert.txt
print(conn.gsql("LS"))

---- Global vertices, edges, and all graphs
Vertex Types:
- VERTEX Specialty(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX SubSpecialty(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Prescriber(PRIMARY_ID Prescriber_id STRING, pageRank FLOAT, communityId INT, articleRank FLOAT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Claim(PRIMARY_ID Claim_id STRING, rx_fill_date DATETIME, ICD10Code STRING, ICD10CodeDescription STRING, CodeGroupTitle STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
- VERTEX Patient(PRIMARY_ID Patient_id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="false"
Edge Types:
- DIRECTED EDGE submitted_by(FROM Claim, TO Prescriber) WITH REVERSE_EDGE="reverse_submitted_by"
- DIRECTED EDGE reverse_submitted_by(FROM Prescriber, TO Claim) WITH REVERSE_EDGE="submitted_by"
- DIRECTED EDGE associa

In [61]:
with open("../../configs/tigergraph_config_1.json", "r") as config:
        args = json.load(config)

conn.getToken(args['gsqlSecret'])

('9ge38iq0q74ip96huuir01uamtv8hspm', 1678311203, '2023-03-08 21:33:23')

In [55]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

{'User': 169791, 'Comment': 3617032, 'Subreddit': 2674474}

In [56]:
# Number of edges for every type
conn.getEdgeCount()

{'interacted_with': 864858,
 'posted': 2671545,
 'replied_to': 2671545,
 'belongs_to': 2671545}

## Feature Engineering

In [45]:
f = conn.gds.featurizer()
f.installAlgorithm("tg_pagerank")
f.installAlgorithm("tg_closeness_cent")
f.installAlgorithm("tg_betweenness_cent")
# f.installAlgorithm("tg_label_prop")
# f.installAlgorithm("tg_maximal_indep_set")

'tg_betweenness_cent'

In [46]:
f.installAlgorithm("tg_degree_cent")

'tg_degree_cent'

In [9]:
# only combination that works on our current data
# how influential a comment is
tg_pagerank_params = {
  "v_type": "Comment",
  "e_type": "replied_to",
  "result_attr": "pagerank",
}
results = pd.json_normalize(f.runAlgorithm("tg_pagerank",tg_pagerank_params)[0]['@@top_scores_heap'])
results

,Vertex_ID,score
0,c1acius,196.52895
1,c18z0z2,170.31886
2,c1adjog,78.08121
3,c18oi9w,67.04869
4,c18aby5,57.40501
...,...,...
95,c1al6vh,18.90866
96,c1asxht,18.89968
97,c196gtu,18.80757
98,c17oi1z,18.79666


In [65]:
tg_pagerank_params = {
  "v_type": "User",
  "e_type": "interacted_with",
  "result_attr": "pagerank",
}
results = pd.json_normalize(f.runAlgorithm("tg_pagerank",tg_pagerank_params)[0]['@@top_scores_heap'])
results

,Vertex_ID,score
0,nonsensical_answer,104.99404
1,andrewsmith1986,102.03746
2,ThatsItGuysShowsOver,68.71323
3,AMerrickanGirl,65.76088
4,moogle516,54.91937
...,...,...
95,powercow,24.21767
96,Iggyhopper,24.16838
97,lackofbrain,24.09249
98,ex_ample,23.89307


In [26]:
tg_degree_params = {
    "v_type": ["User"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"]
}
results = pd.json_normalize(f.runAlgorithm("tg_degree_cent",tg_degree_params)[0]['top_scores'])
results

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Vertex_ID,score
0,andrewsmith1986,4415
1,nonsensical_answer,3573
2,NukeThePope,2498
3,NoMoreNicksLeft,2217
4,AMerrickanGirl,2149
...,...,...
95,inyouraeroplane,946
96,Nebu,945
97,smacksaw,945
98,jamesneysmith,942


In [28]:
tg_degree_params = {
    "v_type": ["Subreddit", "Comment"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"]
}
results = pd.json_normalize(f.runAlgorithm("tg_degree_cent",tg_degree_params)[0]['top_scores'])
results

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Vertex_ID,score
0,AskReddit,365460
1,pics,199496
2,gaming,136544
3,reddit.com,136278
4,politics,118891
...,...,...
95,web_design,3301
96,Anarchism,3253
97,Eve,3246
98,history,3189


In [35]:
tg_degree_params = {
    "v_type": ["Comment"],
    "e_type": ["interacted_with", "posted", "replied_to"]
}
results = pd.json_normalize(f.runAlgorithm("tg_degree_cent",tg_degree_params)[0]['top_scores'])
results

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Vertex_ID,score
0,c1acius,424
1,c18z0z2,385
2,c1adjog,167
3,c18oi9w,150
4,c18aby5,133
...,...,...
95,c18j971,43
96,c1apy2a,42
97,c1asxht,42
98,c1aq25k,42


In [38]:
tg_eigen_params = {
    "v_type": ["User", "Comment", "Subreddit"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"],
}

f.runAlgorithm("tg_eigenvector_cent", params=tg_eigen_params, timeout=600000)

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


HTTPError: 408 Client Error: Timeout for url: https://scott-ucds-hc.i.tgcloud.io:443/restpp/query/Reddit_Interactions/tg_eigenvector_cent

In [50]:
tg_closeness_params = {
    "v_type": ["User", "Comment", "Subreddit"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"],
    "max_hops": 3,
    "top_k": 1
}

f.runAlgorithm("tg_closeness_cent", params=tg_closeness_params, timeout=600000)

[{'top_scores': []}]

In [49]:
tg_betweenness_params = {
    "v_type": ["User", "Comment", "Subreddit"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"],
    "top_k": 1
}

f.runAlgorithm("tg_betweenness_cent", tg_betweenness_params)


[{'top_scores': []}]

In [88]:
tg_label_params = {
    "v_type": ["User", "Comment", "Subreddit"],
    "e_type": ["interacted_with", "posted", "replied_to", "belongs_to"],
    "max_iter": 10,
    "output_limit": 100,
    "attr": "label_prop_score"
}

f.runAlgorithm("tg_label_prop", tg_label_params, timeout=300000)

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


HTTPError: 400 Client Error: Bad Request for url: https://scott-ucds-hc.i.tgcloud.io:443/restpp/query/Reddit_Interactions/tg_label_prop

### Data Loader

In [106]:
# graph_loader = conn.gds.graphLoader(
#     num_batches=10,
#     v_in_feats = {
#         "Comment": ['created_utc', 'subreddit', 'karma'],
#         "User": ['name', 'link_karma', 'comment_karma', 'profile_over_18'],
#         "Subreddit": ['name', 'allow_images', 'allow_videogifs', 'allow_videos', 'created_utc', 'over18', 'subscribers']
#         },
#     v_extra_feats={
#         "User": ["is_train", "is_test"],
#         "Comment": ["is_train", "is_test"],
#         "Subreddit": ["is_train", "is_test"]
#         },
#     e_in_feats={
#         "interacted": ["num_interactions"]
#         },
#     e_extra_feats={
#         "interacted": ["is_train", "is_test"],
#         "wrote": ["is_train", "is_test"],
#         "replied_to": ["is_train", "is_test"],
#         "belongs_to": ["is_train", "is_test"]
#     },
#     output_format = "PyG")

# %time
# graph_loader = conn.gds.graphLoader(
#     num_batches=1,
#     v_in_feats = {
#         "Comment": ['created_utc', 'subreddit', 'karma'],
#         },
#     v_extra_feats={
#         "User": ["is_train", "is_test"],
#         "Comment": ["is_train", "is_test"],
#         "Subreddit": ["is_train", "is_test"]
#         },
#     e_extra_feats={
#         "wrote": ["is_train", "is_test"],
#         "replied_to": ["is_train", "is_test"],
#         "belongs_to": ["is_train", "is_test"]
#     },
#     output_format = "PyG")

Wall time: 0 ns


In [ ]:
# data = graph_loader.data

### Train Test Split

In [73]:
# split vertices
splitter = conn.gds.vertexSplitter(is_train=0.9, is_test=0.1)
splitter.run()

Splitting vertices...
Vertex split finished successfully.


In [74]:
vertices = conn.getVertexDataFrame("User", select="is_train,is_test")

In [75]:
for attr in ["is_train", "is_test"]:
    print("Fraction of vertices with {} = True: {}".format(attr, vertices[attr].sum()/len(vertices)))

Fraction of vertices with is_train = True: 0.9002715102685066
Fraction of vertices with is_test = True: 0.09972848973149342


In [76]:
# split edges
splitter = conn.gds.edgeSplitter(is_train=0.9, is_test=0.1)
splitter.run()

Splitting edges...
Edge split finished successfully.


In [84]:
edges = conn.getEdgesDataFrame(select="is_train, is_test", sourceVertexType="User", sourceVertexId="ihsw", edgeType="posted")

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [87]:
for attr in ["is_train", "is_test"]:
    print("Fraction of edges with {} = True: {}".format(attr, edges[attr].sum()/len(edges)))

Fraction of edges with is_train = True: 0.9880952380952381
Fraction of edges with is_test = True: 0.2261904761904762


### GSQL Queries

In [98]:
# get top 3 highest karma comments a user has made
query = """
USE GRAPH Reddit_Interactions
CREATE QUERY get_top_cmts (Vertex<User> u) SYNTAX v2 {
    start = {u};
    comments = SELECT c
              FROM start:s -(posted:p)- Comment:c
              ORDER BY c.karma ASC
              LIMIT 3;
              
    PRINT comments[comments.subreddit, comments.karma];
}
"""

results = conn.gsql(query)
results

"Using graph 'Reddit_Interactions'\nSuccessfully created queries: [get_top_cmts]."

In [99]:
results = conn.gsql(
'''
USE GRAPH Reddit_Interactions
INSTALL QUERY get_top_cmts
'''
)

In [100]:
parameters = {
    "u": "NozE8"
}
results = conn.runInstalledQuery("get_top_cmts", params=parameters)
print(json.dumps(results, indent=1))

[
 {
  "comments": [
   {
    "v_id": "c1ayyis",
    "v_type": "Comment",
    "attributes": {
     "comments.subreddit": "wow",
     "comments.karma": 0
    }
   },
   {
    "v_id": "c1a7cg4",
    "v_type": "Comment",
    "attributes": {
     "comments.subreddit": "wow",
     "comments.karma": 1
    }
   },
   {
    "v_id": "c1agj7x",
    "v_type": "Comment",
    "attributes": {
     "comments.subreddit": "wow",
     "comments.karma": 1
    }
   }
  ]
 }
]


In [102]:
# get all subreddits a user has made a comment in
query = """
USE GRAPH Reddit_Interactions

CREATE QUERY get_subreddits (Vertex<User> u) SYNTAX v2 {
    start = {u};
    comments = SELECT c
              FROM start:s -(posted:p)- Comment:c;
    
    PRINT comments[comments.subreddit];
}

INSTALL QUERY get_subreddits
"""

results = conn.gsql(query)
results

'Using graph \'Reddit_Interactions\'\nSuccessfully created queries: [get_subreddits].\nStart installing queries, about 1 minute ...\ntg_closeness_cent query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/tg_closeness_cent?v_type_set=VALUE&e_type_set=VALUE&reverse_e_type=VALUE&[max_hops=VALUE]&[top_k=VALUE]&[wf=VALUE]&[print_results=VALUE]&[result_attribute=VALUE]&[file_path=VALUE]&[display_edges=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\ntg_betweenness_cent query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/tg_betweenness_cent?v_type_set=VALUE&e_type_set=VALUE&reverse_e_type=VALUE&[max_hops=VALUE]&[top_k=VALUE]&[print_results=VALUE]&[result_attribute=VALUE]&[file_path=VALUE]&[display_edges=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nget_subreddits query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/get_subreddits?u=VALUE\'. Add -H "Authorization: Bearer TOKEN" if a

In [116]:
def get_subreddits(user):
    parameters = {
        "u": user
    }
    results = conn.runInstalledQuery("get_subreddits", params=parameters)
    subs = set()
    for comment in results[0]['comments']:
        subs.add(comment['attributes']['comments.subreddit'])
    return subs

print(get_subreddits("NozE8"))
print(get_subreddits("Ravelnze"))
print(get_subreddits("Dengar"))

{'IAmA', 'food', 'wow'}
{'wow', 'photography'}
{'IAmA', 'pics', 'politics', 'blog', 'washingtondc', 'worldnews', 'atheism', 'DoesAnybodyElse', 'todayilearned', 'videos', 'geek'}


In [118]:
# get all the subreddits of users that this user has interacted with
query = """
USE GRAPH Reddit_Interactions

CREATE QUERY get_neighbor_subreddits (Vertex<User> u) SYNTAX v2 {
    start = {u};
    users = SELECT tgt
              FROM start:s -(interacted_with:i)- User:tgt;

    comments = SELECT c
                FROM users -(posted:p)- Comment:c;
    
    PRINT comments[comments.subreddit];
}

INSTALL QUERY get_neighbor_subreddits
"""

results = conn.gsql(query)
results

'Using graph \'Reddit_Interactions\'\nSuccessfully created queries: [get_neighbor_subreddits].\nStart installing queries, about 1 minute ...\ntg_closeness_cent query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/tg_closeness_cent?v_type_set=VALUE&e_type_set=VALUE&reverse_e_type=VALUE&[max_hops=VALUE]&[top_k=VALUE]&[wf=VALUE]&[print_results=VALUE]&[result_attribute=VALUE]&[file_path=VALUE]&[display_edges=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\ntg_betweenness_cent query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/tg_betweenness_cent?v_type_set=VALUE&e_type_set=VALUE&reverse_e_type=VALUE&[max_hops=VALUE]&[top_k=VALUE]&[print_results=VALUE]&[result_attribute=VALUE]&[file_path=VALUE]&[display_edges=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nget_neighbor_subreddits query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/get_neighbor_subreddits?u=VALUE\'. Add -H "Author

In [127]:
def get_neighboring_subreddits(user):
    parameters = {
        "u": user
    }
    results = conn.runInstalledQuery("get_neighbor_subreddits", params=parameters)
    subs = set()
    for comment in results[0]['comments']:
        subs.add(comment['attributes']['comments.subreddit'])
    return subs

get_neighboring_subreddits("NozE8")

c:\Users\ryand\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'4chan',
 'AMA',
 'Amateur',
 'Anarchism',
 'Android',
 'AskReddit',
 'Bacon',
 'Bad_Cop_No_Donut',
 'Cooking',
 'DIY',
 'DateRape',
 'DoesAnybodyElse',
 'Drugs',
 'Economics',
 'Favors',
 'Fitness',
 'FoodPorn',
 'Frugal',
 'GetMotivated',
 'Homebrewing',
 'IAmA',
 'IWantOut',
 'MensRights',
 'Military',
 'Music',
 'PHP',
 'TheAgora',
 'WTF',
 'WeAreTheFilmMakers',
 'WeAreTheMusicMakers',
 'animation',
 'askscience',
 'atheism',
 'aviation',
 'aww',
 'blog',
 'books',
 'buildapc',
 'business',
 'cars',
 'circlejerk',
 'comics',
 'entertainment',
 'environment',
 'fffffffuuuuuuuuuuuu',
 'food',
 'funny',
 'gadgets',
 'gaming',
 'geek',
 'guns',
 'hardware',
 'harrypotter',
 'humor',
 'iphone',
 'kindle',
 'lgbt',
 'linux',
 'london',
 'loseit',
 'movies',
 'netsec',
 'news',
 'offbeat',
 'ottawa',
 'photography',
 'pics',
 'politics',
 'reddit.com',
 'science',
 'seduction',
 'sex',
 'shittyadvice',
 'soccer',
 'space',
 'starcraft',
 'sweden',
 'technology',
 'tf2',
 'todayilearned',

In [130]:
query = """
USE GRAPH Reddit_Interactions

CREATE QUERY get_users_subs (Vertex<User> u) SYNTAX v2 {
    ListAccum<STRING> @subs;

    start = {u};
    users = SELECT tgt
            FROM start:s -(interacted_with:i)- User:tgt;

    users_subs = SELECT x
                 FROM users:x -(posted:p)- Comment:tgt
                 ACCUM x.@subs += tgt.subreddit;
    
    PRINT users_subs[users_subs.@subs];
}

INSTALL QUERY get_users_subs
"""

results = conn.gsql(query)
results

'Using graph \'Reddit_Interactions\'\nSuccessfully created queries: [get_users_subs].\nStart installing queries, about 1 minute ...\ntg_betweenness_cent query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/tg_betweenness_cent?v_type_set=VALUE&e_type_set=VALUE&reverse_e_type=VALUE&[max_hops=VALUE]&[top_k=VALUE]&[print_results=VALUE]&[result_attribute=VALUE]&[file_path=VALUE]&[display_edges=VALUE]\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nget_users_subs query: curl -X GET \'https://127.0.0.1:9000/query/Reddit_Interactions/get_users_subs?u=VALUE\'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.\nSelect \'m1\' as compile server, now connecting ...\nNode \'m1\' is prepared as compile server.\n\nQuery installation finished.'

In [171]:
def calc_user_score(user):
    # given a user, calculates how similar they are compared to all of the other users they have interacted with
    # similarity metric is determined by number of subreddits in common / number of subreddits both users are active in 
    parameters = {
        "u": user
    }
    results = conn.runInstalledQuery("get_users_subs", params=parameters)

    my_subs = get_subreddits(user)
    ans = {}
    
    for comment in results[0]['users_subs']:
        other_subs = set(comment['attributes']['users_subs.@subs'])
        total_subs = len(my_subs.union(other_subs))
        score = len(my_subs.intersection(other_subs)) / total_subs
        ans[comment['v_id']] = score
        
    return ans

calc_user_score("Dengar")

{'Vitalstatistix': 0.125,
 'benniaustindev': 0.10526315789473684,
 'burzmali': 0.06666666666666667,
 'zenhob': 0.23076923076923078,
 'HotmailOnreddit': 0.08333333333333333,
 'Midianite_Caller': 0.23076923076923078,
 'tehbizz': 0.19047619047619047,
 'aepr88': 0.23076923076923078,
 'ggggbabybabybaby': 0.1891891891891892,
 'julia_gulia': 0.16666666666666666,
 'devgeek0': 0.21739130434782608,
 'proudblond': 0.07692307692307693,
 'Mizzenmast': 0.08333333333333333,
 'murphymurphy': 0.20833333333333334,
 'BANANARCHY': 0.2,
 'softmaker': 0.23809523809523808,
 'NozE8': 0.07692307692307693,
 'kimjongsick': 0.14285714285714285,
 'itzepiic': 0.3684210526315789,
 'radbro': 0.25,
 'rdjw': 0.07692307692307693}

In [153]:
def diff_subs(user_one, user_two):
    # given user one and two, return subreddits of user_two that user_one isnt apart of
    one_subs = get_subreddits(user_one)
    two_subs = get_subreddits(user_two)
    return two_subs - one_subs

In [182]:
def recommend_subs(user, sim_score=3, num_rec=1):
    # could improve on this by checking how similar the recommended subs are to the current users subs and recommend the most similar subs instead of picking random ones
    user_scores = calc_user_score(user)
    recs = set()

    # third highest similarity score
    threshold = sorted(user_scores.values())[-sim_score:][0]

    for neighbor_user, score in user_scores.items():
        if score >= threshold:
            recs.update(random.sample(diff_subs(user, neighbor_user), num_rec))

    return recs

recommend_subs("jittwoii")

<ipython-input-182-746aaaa4b6f9>:10: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  recs.update(random.sample(diff_subs(user, neighbor_user), num_rec))


{'DoesAnybodyElse', 'Drugs', 'announcements'}